In [10]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_community.chat_models import ChatOllama


In [2]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage], add_messages]


In [11]:
llm=ChatOllama(model="mistral:latest",
    temperature=0.2)

def chat_node(state:ChatState):

    # take user uery state
    messages=state['messages']

    # send to llm
    response=llm.invoke(messages)

    # response store state
    return {'messages':[response]}





In [12]:
## add memory in loop of chat
from langgraph.checkpoint.memory import MemorySaver

In [13]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=checkpointer)

In [14]:
initial_state={
    'messages':[HumanMessage(content='What is capital of india')]
}
# chatbot.invoke(initial_state)['messages'][-1].content

In [15]:
thread_id = "1"

while True:
    user_message = input("Type here: ")

    print('USer:', user_message)

    if user_message.strip().lower() in ["exit", "quit", "bye"]:
        break

    config = {"configurable": {"thread_id": thread_id}}

    response = chatbot.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config=config
    )

    print("AI:", response["messages"][-1].content)

USer: hi iam vikash 
AI:  Hello Vikash! How can I assist you today?
USer: what is my name
AI:  Your name, as per the greeting you provided earlier, is Vikash. Is there something specific you would like to know or discuss related to your name or any other topic?
USer: exit
